# Create the banner for our GitHub repo and docs

In [ ]:
import numpy as np
import birdwatcher as bw
from birdwatcher.movementdetection import detect_movementmog2
%matplotlib inline

we use the example video that comes with birdwatcher

In [ ]:
vfs = bw.testvideosmall()

we apply a movement detection function, based on background subtraction

In [ ]:
coords, coordscount, coordsmean = detect_movementmog2(vfs, morphologyex=1, analysispath='.',
                                                      ignore_firstnframes=20, overwrite=True,
                                                      VarThreshold=200, NMixtures=8, History=3)

`coords` is an array that has the coordinates of the positive pixels for each frame

`coordscount` is an array with the total number of pixels for each frame

`coordsmean` is an array with the mean of pixel coordinates for each frame (we are not using that here)

Let's create the video. We want an animated gif of positive pixels in orange, cropped, with measurement values superimposed on it. You can look all methods up in the documentation: https://birdwatcher.readthedocs.io/en/latest/api.html#module-birdwatcher.frameprocessing

In [ ]:
# a list of strings with the number of suprathreshold pixels
counttextlist = [f'count : {c:4}' for c in coordscount]
# an array of the spatial mean of suprathreshold pixels, excluding frames with count <= 20, which may be random
coordsmean = np.array([coord if c > 20 else (np.nan, np.nan) for (coord,c) in zip(coordsmean,coordscount)])
#  list of strings with spatial mean
meantextlist = [f'mean : ({x:.0f}, {y:.0f})' if not np.isnan(x) else 'mean : ' for (x,y) in coordsmean ]
# difference in distance between adjacent spatial means, needed for speed parameter
diff = [(0,0)] + [(x,y) for (x,y) in zip(np.diff(coordsmean[:,0]),np.diff(coordsmean[:,1]))]
# we include for speed only when many pixels change (>400), which is when bird moves around
speed = [(x**2+y**2)*0.5 if c>400 else np.nan for ((x,y),c) in zip(diff,coordscount) ]
speedtextlist = [f'speed : {s:.0f}' if not np.isnan(s) else 'speed : 'for s in speed]
vfs = (coords.iter_frames(nchannels=3, value=(0,140,255), dtype='uint8')
             .crop(250,450,0,1280)
             .draw_text(counttextlist, org=(1030,25),fontscale=0.8)
             .draw_text(meantextlist, org=(1030,50),fontscale=0.8)
             .draw_text(speedtextlist, org=(1030,75),fontscale=0.8)
             .tovideo('banner_vt200_ex1.gif', format='gif', framerate=25, crf=None, codec=None, pixfmt=None))

In [ ]:
vfs.show()